# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory


# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory

### YOUR CODE HERE ###
from azureml.core.dataset import Dataset
##url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv'
##dataset2 = TabularDatasetFactory.from_delimited_files(path = url)
###ds2 = dataset2.to_pandas_dataframe() ### YOUR CODE HERE ###

url = 'https://raw.githubusercontent.com/zaza107-1/nd00333-capstone/master/heart_failure_clinical_records_dataset.csv'
dataset = Dataset.Tabular.from_delimited_files(path=url)
ds = dataset.to_pandas_dataframe()

ds.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


In [3]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'AutoML_heart'
project_folder = '.'
experiment=Experiment(ws, experiment_name)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FUW4WRL7K to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.

Running


In [5]:
from train2 import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds) ### your code###

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42, test_size=0.33)

df_train = pd.concat([x_train, y_train], axis=1)

In [6]:
df_train.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,200.000000,200.000000,200.000000,200.000000,200.00000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,60.588335,0.420000,582.765000,0.395000,38.59500,0.365000,260130.062100,1.385200,136.620000,0.635000,0.305000,133.735000,0.270000
std,11.287966,0.494797,944.276208,0.490077,11.68003,0.482638,91079.331644,1.019065,4.318431,0.482638,0.461563,77.612973,0.445074
min,40.000000,0.000000,23.000000,0.000000,20.00000,0.000000,62000.000000,0.500000,113.000000,0.000000,0.000000,4.000000,0.000000
25%,52.000000,0.000000,109.000000,0.000000,30.00000,0.000000,203000.000000,0.900000,134.000000,0.000000,0.000000,78.750000,0.000000
50%,60.000000,0.000000,244.500000,0.000000,38.00000,0.000000,256000.000000,1.100000,137.000000,1.000000,0.000000,119.500000,0.000000
75%,67.250000,1.000000,582.000000,1.000000,45.00000,1.000000,304250.000000,1.400000,140.000000,1.000000,1.000000,206.250000,1.000000
max,95.000000,1.000000,7861.000000,1.000000,80.00000,1.000000,621000.000000,9.000000,146.000000,1.000000,1.000000,285.000000,1.000000


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
compute_target

Name,Workspace,State,Location,VmSize,Application URI,Docs
auto-ml,quick-starts-ws-138715,Running,southcentralus,STANDARD_DS3_V2,Jupyter JupyterLab RStudio,Doc


In [8]:
if 'training' not in os.listdir():
    os.mkdir("./training")

In [9]:
# TODO: Put your automl settings here
automl_settings = {"experiment_timeout_minutes": 15,
                    "max_concurrent_iterations": 5,
                    "primary_metric" : 'accuracy'}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings)

In [10]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

remote_run.wait_for_completion()

Running on remote.


{'runId': 'AutoML_78f226cc-243e-49aa-8e99-1054944e70a5',
 'target': 'auto-ml',
 'status': 'Completed',
 'startTimeUtc': '2021-02-12T17:24:03.931838Z',
 'endTimeUtc': '2021-02-12T17:38:05.249391Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'auto-ml',
  'AMLSettingsJsonString': '{"path":null,"name":"AutoML_heart","subscription_id":"3e42d11f-d64d-4173-af9b-12ecaa1030b3","resource_group":"aml-quickstarts-138715","workspace_name":"quick-starts-ws-138715","region":"southcentralus","compute_target":"auto-ml","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":null,"y_min":null,"y_max":null,"num_classes":null,"featurizat

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [11]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [12]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


Run(Experiment: AutoML_heart,
Id: AutoML_78f226cc-243e-49aa-8e99-1054944e70a5_38,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                subsample=0.24789473684210528,
                                                                                                subsample_for_bin=200000,
                 

In [13]:
#TODO: Save the best model

best_run.register_model(model_name = 'automl_best_model.pkl', model_path = '/outputs/model.pkl')

import joblib

joblib.dump(value= fitted_model, filename = './model_ML_fitted.joblib')

['./model_ML_fitted.joblib']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [24]:
#best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')

from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

env = Environment.get(ws, "AzureML-AutoML")
for pip_package in ["scikit-learn"]:
    env.python.conda_dependencies.add_pip_package(pip_package)

#inference_config = InferenceConfig(entry_script='score.py',
#                                    environment=env)



In [15]:
best_run.properties['model_name']

'AutoML78f226cc238'

In [23]:
#Model.get_model_path('model_ML_fitted.joblib')

In [41]:
from azureml.core import Model
from azureml.core.webservice import AciWebservice, Webservice
import joblib
##model = best_run.register_model(model_name='model_name', model_path='./outputs/model_name.pkl',
  ##                     model_framework=Model.Framework.SCIKITLEARN)
inference_config = InferenceConfig(entry_script='score.py',
                                    environment=env)

model = best_run.register_model(model_name = 'automl_best_model.pkl', model_path = './outputs/model.pkl', model_framework=Model.Framework.SCIKITLEARN )


aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=2,
                                                enable_app_insights=True, auth_enabled=False)  
service_name =  'automl57'
service = Model.deploy(ws, service_name, [model], inference_config, deployment_config=aci_config, overwrite=True)
service.wait_for_deployment(show_output = True)
print(service.get_logs())

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..............
Succeeded
ACI service creation operation finished, operation "Succeeded"
2021-02-12T19:23:21,390194100+00:00 - gunicorn/run 
2021-02-12T19:23:21,392727000+00:00 - iot-server/run 
2021-02-12T19:23:21,402167500+00:00 - rsyslog/run 
2021-02-12T19:23:21,419443600+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_09ff55f546b313bb1ab136a466214499/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_09ff55f546b313bb1ab136a466214499/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_09ff55f546b313bb1ab136a466214499/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_09ff55

In [ ]:
#print(service.get_logs())

TODO: In the cell below, send a request to the web service you deployed to test it.

In [43]:
import requests
import json
# URL for the web service, should be similar to:
# 'http://8530a665-66f3-49c8-a953-b82a2d312917.eastus.azurecontainer.io/score'
scoring_uri = service.scoring_uri
# If the service is authenticated, set the key or token
#key = ''

# Two sets of data to score, so we get two results back
data = {"data":
        [
          {
           "age": 75, 
           "anaemia": 0, 
           "creatinine_phosphokinase": 582, 
           "diabetes": 0, 
           "ejection_fraction": 20, 
           "high_blood_pressure": 1, 
           "platelets": 265000, 
           "serum_creatinine": 1.9, 
           "serum_sodium": 130, 
           "sex": 1, 
           "smoking": 0,
           "time": 4
          },
          
      ]
    }

In [44]:
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)
# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
#headers['Authorization'] = f'Bearer {key}'
# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())

[1]


In [34]:
resp


<Response [200]>

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
service.delete()
model.delete()